In [0]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.udf import *

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

df1 = spark.read.csv('dbfs:/FileStore/tables/Real_Estate_Sales_updated.csv' , inferSchema=True , header=True)
df2 = spark.read.csv('dbfs:/FileStore/tables/Real_Estate_Sales_updated.csv' , inferSchema=True , header=True)

df1 = df1.drop('Non Use Code' , 'Assessor Remarks', 'OPM remarks', 'Location')
df2 = df2.drop('Non Use Code' , 'Assessor Remarks', 'OPM remarks', 'Location')

df1 = df1.fillna({'Serial Number': 0 , 'List Year': 0 , 'Date Recorded': '01-01-2000' , 'Town': 'NA' , 'Address': 'NA' , 'Assessed Value': 0 , 
           'Sale Amount': 0 , 'Sales Ratio': 0 , 'Property Type': 'NA' , 'Residential Type': 'NA'})
df2 = df2.fillna({'Serial Number': 0 , 'List Year': 0 , 'Date Recorded': '01-01-2000' , 'Town': 'NA' , 'Address': 'NA' , 'Assessed Value': 0 , 
                  'Sale Amount': 0 , 'Sales Ratio': 0 , 'Property Type': 'NA' , 'Residential Type': 'NA'})

joined_df = df1.join(df2 , df1['Serial Number'] == df2['Serial Number'] , 'left').select(df1['Serial Number'] , df1['List Year'] , df1['Date Recorded'] , df1['Town'] , 
                                                                             df1['Address'] , df1['Assessed Value'] , df1['Sale Amount'] , df2['Sales Ratio'] ,
                                                                             df2['Property Type'] , df2['Residential Type'])

filtered_df = joined_df.filter(joined_df['Assessed Value'] > 100000)

on_groupby = filtered_df.groupby('List Year' , 'Town' , 'Property Type' , 'Residential Type').agg({'Town' : 'count'}).orderBy(desc('List Year'))


def len_cal(x):
    return len(x)

len_cal = udf(len_cal , StringType())

new_col = on_groupby.withColumn('Length of Town' , len_cal(col('Town')))

display(new_col)

List Year,Town,Property Type,Residential Type,count(Town),Length of Town
2020,Darien,Residential,Condo,3969,6
2020,Watertown,NA,NA,26990,9
2020,Plainfield,Apartments,NA,49,10
2020,New Britain,Industrial,NA,27,11
2020,Chester,Residential,Condo,2651,7
2020,Avon,Residential,Two Family,2422,4
2020,Bristol,Residential,Single Family,64244,7
2020,East Windsor,Vacant Land,NA,1986,12
2020,New Fairfield,Residential,Single Family,22664,13
2020,Preston,Residential,Three Family,92,7


In [0]:
filtered_df.count()

38743100